# Value proposition of norfair

Norfair is a customizable lightweight Python library for real-time multi-object tracking.
Using Norfair, you can add tracking capabilities to any detector with just a few lines of code.

It means you won't need a SOTA Tracker you can use a basic Tracker with a Kalmann Filter and add the custom logic you want.

# Imports and setup

In [ ]:
import sys; sys.path.append('..')

from lib.norfair_helper.utils import yolo_to_norfair_detection
from lib.sequence import Sequence

import numpy as np
import os
from norfair import Tracker, draw_boxes


If you want to test this code on your detection and frames you can use the following code if you structure the data as follows:

```
data/
   ├── detection/
   │   └── sequence_1/
   │       └── detections_1.txt
   └── frames/
       └── sequence_1/
           └── frame_1.jpg
```

Where the detections.txt file is in the following format scaled between 0 and 1:

```
class_id x_center y_center width height confidence
```

If this is not the case, you'll need to adapt this code to your data.

In [ ]:
DATA_PATH = "../data"
DETECTION_PATH = f"{DATA_PATH}/detections"
FRAME_PATH = f"{DATA_PATH}/frames"
VIDEO_OUTPUT_PATH = "private"

SEQUENCES = os.listdir(FRAME_PATH)

In [ ]:
def get_sequence_frames(sequence):
    frames = os.listdir(f"{FRAME_PATH}/{sequence}")
    frames = [os.path.join(f"{FRAME_PATH}/{sequence}", frame) for frame in frames]
    frames.sort()
    return frames

def get_sequence_detections(sequence):
    detections = os.listdir(f"{DETECTION_PATH}/{sequence}")
    detections = [os.path.join(f"{DETECTION_PATH}/{sequence}", detection) for detection in detections]
    detections.sort()
    return detections

frame_path = get_sequence_frames(SEQUENCES[0])
detection_path = get_sequence_detections(SEQUENCES[0])
test_sequence = Sequence(frame_path, detection_path)
test_sequence

# Basic Usage of Norfair

## Tracker

Norfair tracker object is the customizable object that will track detections.
Norfair expects a distance function that will serve as a metric to match objects between each detection. You can create your own distance metric or use one of the built-in ones such as euclidian distance, iou or many more.

In [ ]:
# Initialize a tracker with the distance function
basic_tracker = Tracker(
    distance_function="mean_euclidean",
    distance_threshold=40,
)

## Basic tracking

In [ ]:
import cv2

FRAME_SIZE = (2560, 1440)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Changed codec to 'mp4v' for compatibility with Mac
out = cv2.VideoWriter(f'{VIDEO_OUTPUT_PATH}/basic_tracking.mp4', fourcc, 20.0, FRAME_SIZE) # Changed file extension to .mp4

for frame, detection in test_sequence:
    detections_list = yolo_to_norfair_detection(detection, frame.size)
    tracked_objects = basic_tracker.update(detections=detections_list)
    frame_detected = draw_boxes(np.array(frame), tracked_objects, draw_ids=True, color="by_label")
    frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_BGR2RGB)
    out.write(frame_detected)
out.release()


## Advanced tracking

In [ ]:
def reid_distance_advanced(new_object, unmatched_object):
    return 0 # ALWAYS MATCH

In [ ]:
advanced_tracker = Tracker(
    distance_function="sqeuclidean",
    distance_threshold=350, # Higher value means objects further away will be matched
    initialization_delay=15, # Wait 15 frames before an object is starts to be tracked
    hit_counter_max=12, # Inertia, higher values means an object will take time to enter in reid phase
    reid_distance_function=reid_distance_advanced, # function to decide on which metric to reid
    reid_distance_threshold=0.5, # If the distance is below 0.5 the object is matched
    reid_hit_counter_max=200, # inertia, higher values means an object will enter reid phase longer
    )

In [ ]:
import cv2

FRAME_SIZE = (2560, 1440)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Changed codec to 'mp4v' for compatibility with Mac
out = cv2.VideoWriter(f'{VIDEO_OUTPUT_PATH}/advance_tracking.mp4', fourcc, 20.0, FRAME_SIZE) # Changed file extension to .mp4

for frame, detection in test_sequence:
    detections_list = yolo_to_norfair_detection(detection, frame.size)
    tracked_objects = advanced_tracker.update(detections=detections_list)
    frame_detected = draw_boxes(np.array(frame), tracked_objects, draw_ids=True, color="by_label")
    frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_BGR2RGB)
    out.write(frame_detected)
out.release()
